In [ ]:
import os
import time
from datetime import datetime
from google.colab import files
from google.colab import drive
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.dates import date2num
plt.style.use('ggplot')
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# look at the bigger picture, don't get lost in the code

<h1> take target_dates of QQQ that have pricing values

In [ ]:
pricing = pd.read_feather('/content/drive/MyDrive/DSO_585_Fall_2023/Monthly_Option_Backtest/03_strategies/straddles.feather')

In [ ]:
pricing.head()

,effexdate,secid,date,exdate,call delta,call vega,call mid,call pctspread,call impl_volatility,call open_interest,...,biasadj dynamic hedged put return,call weight,put weight,pctspread,unhedged straddle return,dynamic hedged straddle return,dynamic spy hedged straddle return,biasadj dynamic spy hedged straddle return,biasadj dynamic hedged straddle return,normvega
0,1996-02-16,101121.0,1996-01-19,1996-02-17,0.337476,1.884713,0.53125,0.117647,0.494899,3287.0,...,-0.426187,0.657237,0.342763,0.099436,-1.004246,-0.754097,-1.728337,-1.661999,-0.979078,2.669580
1,1996-02-16,101310.0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1996-02-16,101594.0,1996-01-19,1996-02-17,0.548319,3.277043,2.87500,0.086957,0.872408,4520.0,...,-0.213609,0.458670,0.541330,0.100032,-0.402768,-0.332520,-0.351441,-0.402550,-0.311145,1.313122
3,1996-02-16,102265.0,1996-01-19,1996-02-17,0.402556,8.364623,1.43750,0.173913,0.241308,1087.0,...,-0.119962,0.599395,0.400605,0.153296,-0.326670,-0.218132,-0.246419,-0.100795,-0.124454,4.574415
4,1996-02-16,103823.0,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pricing.columns.to_list()

['effexdate',
 'secid',
 'date',
 'exdate',
 'call delta',
 'call vega',
 'call mid',
 'call pctspread',
 'call impl_volatility',
 'call open_interest',
 'unhedged call return',
 'static hedged call return',
 'dynamic hedged call return',
 'dynamic spy hedged call return',
 'biasadj dynamic spy hedged call return',
 'biasadj dynamic hedged call return',
 'put delta',
 'put vega',
 'put mid',
 'put pctspread',
 'put impl_volatility',
 'put open_interest',
 'unhedged put return',
 'static hedged put return',
 'dynamic hedged put return',
 'dynamic spy hedged put return',
 'biasadj dynamic spy hedged put return',
 'biasadj dynamic hedged put return',
 'call weight',
 'put weight',
 'pctspread',
 'unhedged straddle return',
 'dynamic hedged straddle return',
 'dynamic spy hedged straddle return',
 'biasadj dynamic spy hedged straddle return',
 'biasadj dynamic hedged straddle return',
 'normvega']

In [ ]:
names = pd.read_feather('/content/drive/MyDrive/DSO_585_Fall_2023/Monthly_Option_Backtest/01_underlying/keep_names.feather')
names = names[['secid','ticker']]

names[names['ticker'] == 'QQQ']


,secid,ticker
21,107899,QQQ


In [ ]:
pricing_2 = pricing[pricing['secid'] == 107899.0]

In [ ]:
pricing_2 = pricing_2[pricing_2['exdate'] > '2019-01-01']


In [ ]:
dates = pricing_2['exdate'].to_list()
dates

[Timestamp('2019-01-18 00:00:00'),
 Timestamp('2019-02-15 00:00:00'),
 Timestamp('2019-03-15 00:00:00'),
 Timestamp('2019-04-18 00:00:00'),
 Timestamp('2019-05-17 00:00:00'),
 Timestamp('2019-06-21 00:00:00'),
 Timestamp('2019-07-19 00:00:00'),
 Timestamp('2019-08-16 00:00:00'),
 Timestamp('2019-09-20 00:00:00'),
 Timestamp('2019-10-18 00:00:00'),
 Timestamp('2019-11-15 00:00:00'),
 Timestamp('2019-12-20 00:00:00'),
 Timestamp('2020-01-17 00:00:00'),
 Timestamp('2020-02-21 00:00:00'),
 Timestamp('2020-03-20 00:00:00'),
 Timestamp('2020-04-17 00:00:00'),
 Timestamp('2020-05-15 00:00:00'),
 Timestamp('2020-06-19 00:00:00'),
 Timestamp('2020-07-17 00:00:00'),
 Timestamp('2020-08-21 00:00:00'),
 Timestamp('2020-09-18 00:00:00'),
 Timestamp('2020-10-16 00:00:00'),
 Timestamp('2020-11-20 00:00:00'),
 Timestamp('2020-12-18 00:00:00')]

In [ ]:
target_dates = []
for i in dates:
   date_string = i.strftime('%Y-%m-%d')
   target_dates.append(date_string)

In [ ]:
target_dates

['2019-01-18',
 '2019-02-15',
 '2019-03-15',
 '2019-04-18',
 '2019-05-17',
 '2019-06-21',
 '2019-07-19',
 '2019-08-16',
 '2019-09-20',
 '2019-10-18',
 '2019-11-15',
 '2019-12-20',
 '2020-01-17',
 '2020-02-21',
 '2020-03-20',
 '2020-04-17',
 '2020-05-15',
 '2020-06-19',
 '2020-07-17',
 '2020-08-21',
 '2020-09-18',
 '2020-10-16',
 '2020-11-20',
 '2020-12-18']

In [ ]:
# files = os.listdir('/content/drive/MyDrive/DSO_585_Fall_2023/Input_Data/Features_Per_Ticker_Per_Day/QQQ_86755')
# files.sort()
# files = files[1000:1500]
# files

In [ ]:
# #upload training data

# x_train = pd.read_feather('/content/drive/MyDrive/DSO_585_Fall_2023/Input_Data/Features_Per_Ticker_Per_Day/QQQ_86755/2012-12-13/X_train_features.feather')
# x_train.rename(columns = {'index': 'Date'}, inplace = True)
# x_train.set_index('Date', inplace = True)
# x_train

<h2> look for Implied vol, predictor for baseline model, we choose the IV of Nasdaq as it is the closest representation to QQQ

In [ ]:
# iv_variables = [col for col in x_train.columns if "IV" in col]
# iv_variables



In [ ]:
# path = '/content/drive/MyDrive/DSO_585_Fall_2023/Input_Data/Features_Per_Ticker_Per_Day/QQQ_86755/'

In [ ]:
#upload test data

#initialize an empty df to store appended results

# x_test_combined = pd.DataFrame()

# for file in files:
#   x_test = pd.read_feather(path + file + '/X_test_features.feather')
#   x_test = x_test.set_index(x_test.columns[0])
#   x_test.index.name = 'Date'
#   x_test = x_test.T
#   x_test.rename({'values': file}, inplace=True)
#   x_test_combined = x_test_combined.append(x_test)
#   x_test_combined.index = pd.to_datetime(x_test_combined.index)


#show combined df
# x_test_combined

In [ ]:
# print(x_test_combined.index.dtype)

In [ ]:
#choosing the predictor IV of Nasdaq for baseline model

# x_test_combined_50 = x_test_combined[['IV_NDX_Call_ATM_30D_CLOSE_lag1d']]
# x_test_combined_50

In [ ]:
#load target variable

# target = pd.read_csv('/content/drive/MyDrive/DSO_585_Fall_2023/Input_Data/Regression_Variables_Minus_Macroecon/QQQ_86755_regression_variables_minus_macroecon.csv', parse_dates=[0])
# target.rename(columns={'Unnamed: 0':'Date'}, inplace=True)
# # target.set_index('Date', inplace=True)


In [ ]:
# y = target[['rv_22d_lead(22+1)d']]
# y

In [ ]:
# print(y.index.dtype)

In [ ]:
#merge the target variable on test dataset

# x_test_combined_50 = x_test_combined_50.merge(y, left_index = True, right_index= True, how = 'left')
# x_test_combined_50

In [ ]:
#join y on x_train on the index
# x_train = x_train.merge(y, left_index=True, right_index=True, how='inner')
# x_train

<h3> Loading train and test data for target dates </h3>
<h3> Forecast IV using Linear Regression </h3>

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
files = os.listdir('/content/drive/MyDrive/DSO_585_Fall_2023/Input_Data/Features_Per_Ticker_Per_Day/QQQ_86755')
files.sort()

In [ ]:
counter = 0

for i in files:
  print(i)
  counter += 1

  if counter == 5:
    break


2008-12-23
2008-12-24
2008-12-26
2008-12-29
2008-12-30


In [ ]:
# path of file in dir
path = '/content/drive/MyDrive/DSO_585_Fall_2023/Input_Data/Features_Per_Ticker_Per_Day/QQQ_86755/'


#target
target = pd.read_csv('/content/drive/MyDrive/DSO_585_Fall_2023/Input_Data/Regression_Variables_Minus_Macroecon/QQQ_86755_regression_variables_minus_macroecon.csv', parse_dates=[0])
target.rename(columns={'Unnamed: 0':'Date'}, inplace=True)
target.set_index('Date', inplace=True)

y = target[['rv_22d_lead(22+1)d']]


<h3> Predictions

In [ ]:
#initialize an empty df to store appended results

x_test_combined = pd.DataFrame()

for file in target_dates:
  x_test = pd.read_feather(path + file + '/X_test_features.feather')
  x_test = x_test.set_index(x_test.columns[0])
  x_test.index.name = 'Date'
  x_test = x_test.T
  x_test.rename({'values': file}, inplace=True)
  x_test_combined = x_test_combined.append(x_test)
  x_test_combined.index = pd.to_datetime(x_test_combined.index)


#only select the IV of Nasdaq as IV

x_test_combined_50 = x_test_combined[['IV_NDX_Call_ATM_30D_CLOSE_lag1d']]

#merge the target variable on test dataset

x_test_combined_50 = x_test_combined_50.merge(y, left_index = True, right_index= True, how = 'left')
x_test_combined_50


,IV_NDX_Call_ATM_30D_CLOSE_lag1d,rv_22d_lead(22+1)d
2019-01-18,420.2500,0.002054
2019-02-15,282.2400,0.001207
2019-03-15,207.9364,0.001229
2019-04-18,190.7161,0.003134
2019-05-17,276.2244,0.003059
2019-06-21,280.8976,0.001160
2019-07-19,226.5025,0.004943
2019-08-16,520.2961,0.002452
2019-09-20,253.1281,0.002228
2019-10-18,243.0481,0.000663


In [ ]:
#initialize an empty list to store error data and actual_predicted_data to store actual and predicted IV

error_data = []

actual_predicted_data = []


#upload train data

for file in target_dates:
  x_train = pd.read_feather(path + file + '/X_train_features.feather')
  x_train.rename(columns = {'index': 'Date'}, inplace = True)
  x_train.set_index('Date', inplace = True)


  #merging

  x_train = x_train.merge(y, left_index=True, right_index=True, how='inner')

  #model
  X_Train = x_train[['IV_NDX_Call_ATM_30D_CLOSE_lag1d']]
  Y_Train = x_train['rv_22d_lead(22+1)d']

  X_Test = x_test_combined_50[x_test_combined_50.index == file][['IV_NDX_Call_ATM_30D_CLOSE_lag1d']]
  Y_Actual = x_test_combined_50[x_test_combined_50.index == file]['rv_22d_lead(22+1)d']
  model = LinearRegression()

  model.fit(X_Train, Y_Train)

  Y_Pred = model.predict(X_Test)

  # Print out the prediction
  #print(f"The predicted y value is: {Y_Pred[0]}")

  # rmse

  rmse = np.sqrt(mean_squared_error(Y_Actual, Y_Pred))

  # r squared
  r_squared = r2_score(Y_Actual, Y_Pred)

  # Append the error data for this iteration into a list of dictionaries
  error_data.append({'Date': file, 'RMSE': rmse, 'R_Squared': r_squared})

  #append to new_df actual and predicted IV values
  actual_predicted_data.append({'Date': file, 'Actual': Y_Actual.values, 'Predicted': Y_Pred})

# Create a DataFrame from the collected error data after the loop
error_values_df = pd.DataFrame(error_data)
actual_predicted_df = pd.DataFrame(actual_predicted_data)






In [ ]:
error_values_df

,Date,RMSE,R_Squared
0,2019-01-18,0.002106,NaN
1,2019-02-15,0.001756,NaN
2,2019-03-15,0.000998,NaN
3,2019-04-18,0.001166,NaN
4,2019-05-17,0.000340,NaN
5,2019-06-21,0.001629,NaN
6,2019-07-19,0.002694,NaN
7,2019-08-16,0.002213,NaN
8,2019-09-20,0.000243,NaN
9,2019-10-18,0.001729,NaN


In [ ]:
actual_predicted_df

,Date,Actual,Predicted
0,2019-01-18,[0.0020540806949999],[0.0041602740240818115]
1,2019-02-15,[0.0012071395289999],[0.002962819433506574]
2,2019-03-15,[0.0012286967329999],[0.0022269758536958223]
3,2019-04-18,[0.0031338113329999],[0.001967947121755861]
4,2019-05-17,[0.0030585755549999],[0.0027186462645716657]
5,2019-06-21,[0.0011598280969999],[0.002789234521538723]
6,2019-07-19,[0.004942721987],[0.002248461400523516]
7,2019-08-16,[0.0024516533859999],[0.004664661478303991]
8,2019-09-20,[0.0022276334729999],[0.002470404749003636]
9,2019-10-18,[0.0006634164859999],[0.002392754844963086]


In [ ]:
actual_predicted_df.set_index('Date', inplace = True)

In [ ]:
actual_predicted_df

,Actual,Predicted
Date,,
2019-01-18,[0.0020540806949999],[0.0041602740240818115]
2019-02-15,[0.0012071395289999],[0.002962819433506574]
2019-03-15,[0.0012286967329999],[0.0022269758536958223]
2019-04-18,[0.0031338113329999],[0.001967947121755861]
2019-05-17,[0.0030585755549999],[0.0027186462645716657]
2019-06-21,[0.0011598280969999],[0.002789234521538723]
2019-07-19,[0.004942721987],[0.002248461400523516]
2019-08-16,[0.0024516533859999],[0.004664661478303991]
2019-09-20,[0.0022276334729999],[0.002470404749003636]


In [ ]:
pricing_2 = pricing_2.set_index('exdate')


In [ ]:
pricing_2

,effexdate,secid,date,call delta,call vega,call mid,call pctspread,call impl_volatility,call open_interest,unhedged call return,...,biasadj dynamic hedged put return,call weight,put weight,pctspread,unhedged straddle return,dynamic hedged straddle return,dynamic spy hedged straddle return,biasadj dynamic spy hedged straddle return,biasadj dynamic hedged straddle return,normvega
exdate,,,,,,,,,,,,,,,,,,,,,
2019-01-18,2019-01-18,107899.0,2018-12-21,0.500196,16.26066,5.110,0.011742,0.330798,1194.0,2.373858,...,-0.122799,0.499955,0.500045,0.011266,0.685840,0.164779,0.095841,-0.092601,-0.113285,3.052318
2019-02-15,2019-02-15,107899.0,2019-01-18,0.490962,18.25596,3.275,0.009160,0.190642,9414.0,0.811865,...,-0.061835,0.509932,0.490068,0.007128,-0.076991,-0.090520,-0.320830,-0.286059,-0.030807,5.080597
2019-03-15,2019-03-15,107899.0,2019-02-15,0.523009,18.92622,3.170,0.006309,0.159818,30767.0,1.001279,...,0.009440,0.487919,0.512081,0.006481,-0.024498,-0.186310,-0.064288,0.039859,-0.017361,6.137618
2019-04-18,2019-04-18,107899.0,2019-03-15,0.495126,21.71562,2.735,0.010969,0.131099,10614.0,2.065361,...,-0.097580,0.506392,0.493608,0.009792,0.551149,-0.068241,0.340787,0.262897,-0.075579,7.078769
2019-05-17,2019-05-17,107899.0,2019-04-18,0.494151,21.07142,2.695,0.011132,0.133337,34860.0,-1.001940,...,0.392210,0.509065,0.490935,0.010584,-0.188905,0.419001,0.246645,0.275781,0.402066,7.421567
2019-06-21,2019-06-21,107899.0,2019-05-17,0.490637,22.53922,3.810,0.005249,0.180007,25583.0,0.197158,...,0.329582,0.516109,0.483891,0.007200,-0.383256,-0.018512,-0.020006,0.073970,0.236877,5.582694
2019-07-19,2019-07-19,107899.0,2019-06-21,0.486988,20.78212,3.400,0.005882,0.174961,4503.0,-0.410674,...,-0.370103,0.514726,0.485274,0.008109,-0.697557,-0.424473,-0.486445,-0.417259,-0.418016,5.778485
2019-08-16,2019-08-16,107899.0,2019-07-19,0.476273,21.06829,2.905,0.010327,0.152317,31776.0,-1.001720,...,0.391300,0.525727,0.474273,0.009311,-0.157994,0.571070,0.354536,0.297621,0.497575,6.531160
2019-09-20,2019-09-20,107899.0,2019-08-16,0.507991,22.88276,4.575,0.006557,0.203074,9474.0,0.047141,...,-0.239605,0.499371,0.500629,0.006487,-0.478109,-0.130808,-0.360878,-0.452352,-0.260642,4.945657


In [ ]:
actual_predicted_df['Call Mid'] = pricing_2['call mid']
actual_predicted_df['Put Mid'] = pricing_2['put mid']

Tally for adding columns from pricing_2 to actual_predicted_df

In [ ]:
actual_predicted_df.head(6)

,Actual,Predicted,Call Mid,Put Mid
Date,,,,
2019-01-18,[0.0020540806949999],[0.0041602740240818115],5.110,5.560
2019-02-15,[0.0012071395289999],[0.002962819433506574],3.275,3.990
2019-03-15,[0.0012286967329999],[0.0022269758536958223],3.170,3.010
2019-04-18,[0.0031338113329999],[0.001967947121755861],2.735,3.495
2019-05-17,[0.0030585755549999],[0.0027186462645716657],2.695,2.995
2019-06-21,[0.0011598280969999],[0.002789234521538723],3.810,4.310


In [ ]:
pricing_2[['call mid', 'put mid']].head(6)

,call mid,put mid
exdate,,
2019-01-18,5.110,5.560
2019-02-15,3.275,3.990
2019-03-15,3.170,3.010
2019-04-18,2.735,3.495
2019-05-17,2.695,2.995
2019-06-21,3.810,4.310


<h2> Random Forest

<h2>Xg boost

<h2> exponential smoothing

In [ ]:
efrom statsmodels.tsa.holtwinters import ExponentialSmoothing

# Your code for loading data and setting up the files list remains the same

error_data = []

for file in files:
  x_train = pd.read_feather(path + file + '/X_train_features.feather')
  x_train.rename(columns = {'index': 'Date'}, inplace = True)
  x_train.set_index('Date', inplace = True)


  #merging

  x_train = x_train.merge(y, left_index=True, right_index=True, how='inner')

  #model
  X_Train = x_train[['IV_NDX_Call_ATM_30D_CLOSE_lag1d']]
  Y_Train = x_train['rv_22d_lead(22+1)d']

  X_Test = x_test_combined_50[['IV_NDX_Call_ATM_30D_CLOSE_lag1d']]
  Y_Actual = x_test_combined_50['rv_22d_lead(22+1)d']


    # Train Exponential Smoothing model
  model = ExponentialSmoothing(Y_Train, trend='add', seasonal=None, seasonal_periods=None)
  model_fit = model.fit(optimized=True)

    # Make predictions
  Y_Pred = model_fit.forecast(len(X_Test))

    # Calculate RMSE and R-squared
  rmse = np.sqrt(mean_squared_error(Y_Actual, Y_Pred))
  r_squared = r2_score(Y_Actual, Y_Pred)

    # Append the error data
  error_data.append({
        'Date': file,
        'RMSE': rmse,
        'R_Squared': r_squared
    })

error_values_df = pd.DataFrame(error_data)